# LangGraph Music Router with Azure AI Telemetry

Stream a LangGraph workflow that routes tool calls while `langchain-azure-ai` records GenAI-compliant spans for each model invocation and tool execution.

## Requirements
Install these packages before running the notebook.

```bash
pip install langchain langgraph langchain-openai langchain-azure-ai python-dotenv
pip install azure-identity  # required when API_HOST=azure
```

Set `API_HOST` to `github` (default) or `azure`, provide the corresponding credentials, and optionally `APPLICATION_INSIGHTS_CONNECTION_STRING` for Azure Monitor export.

### Step 1: Import LangGraph, LangChain, and telemetry helpers
Bring in the required libraries and load environment variables for your chosen provider.


In [ ]:
from __future__ import annotations

import os

import azure.identity
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI, ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode

from langchain_azure_ai.callbacks.tracers import AzureAIOpenTelemetryTracer

load_dotenv(override=True)

### Step 2: Configure the chat model and tracer
Initialise the chat client (GitHub Models or Azure OpenAI) and the Azure AI tracer so spans are emitted for every model call.


In [ ]:
def _build_model():
    host = os.getenv("API_HOST", "github").lower()

    if host == "azure":
        token_provider = azure.identity.get_bearer_token_provider(
            azure.identity.DefaultAzureCredential(),
            "https://cognitiveservices.azure.com/.default",
        )
        return AzureChatOpenAI(
            azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
            azure_deployment=os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT"),
            openai_api_version=os.environ.get("AZURE_OPENAI_VERSION"),
            azure_ad_token_provider=token_provider,
        )

    if host == "github":
        return ChatOpenAI(
            model=os.getenv("GITHUB_MODEL", "gpt-4o"),
            base_url=os.getenv("GITHUB_OPENAI_BASE_URL", "https://models.inference.ai.azure.com"),
            api_key=os.environ.get("GITHUB_TOKEN"),
        )

    raise ValueError("API_HOST must be 'github' or 'azure'")


TRACER = AzureAIOpenTelemetryTracer(
    connection_string=os.environ.get("APPLICATION_INSIGHTS_CONNECTION_STRING"),
    enable_content_recording=os.getenv("OTEL_RECORD_CONTENT", "true").lower() == "true",
    name="Music Player Agent",
)

MODEL = _build_model()
print("Model ready:", MODEL.model_name if hasattr(MODEL, "model_name") else "custom")

### Step 3: Declare tools and bind them to the model
Define the music playback tools and bind them to the chat model so LangGraph can leverage them during execution.


In [ ]:
@tool
def play_song_on_spotify(song: str) -> str:
    """Simulated Spotify playback."""

    return f"Successfully played {song} on Spotify!"


@tool
def play_song_on_apple(song: str) -> str:
    """Simulated Apple Music playback."""

    return f"Successfully played {song} on Apple Music!"


TOOLS = [play_song_on_spotify, play_song_on_apple]
TOOL_NODE = ToolNode(TOOLS)
MODEL_WITH_TOOLS = MODEL.bind_tools(TOOLS, parallel_tool_calls=False)


### Step 4: Build the LangGraph workflow
Create the agent and tool nodes, wire up conditional edges, and compile the workflow with an in-memory checkpointer.


In [ ]:
def should_continue(state: MessagesState) -> str:
    messages = state["messages"]
    last_message = messages[-1]
    return "end" if not last_message.tool_calls else "continue"


def call_model(state: MessagesState) -> dict:
    messages = state["messages"]
    response = MODEL_WITH_TOOLS.invoke(messages)
    return {"messages": [response]}


WORKFLOW = StateGraph(MessagesState)
WORKFLOW.add_node("agent", call_model)
WORKFLOW.add_node("action", TOOL_NODE)
WORKFLOW.add_edge(START, "agent")
WORKFLOW.add_conditional_edges("agent", should_continue, {"continue": "action", "end": END})
WORKFLOW.add_edge("action", "agent")

MEMORY = MemorySaver()
APP = WORKFLOW.compile(checkpointer=MEMORY)
print("Workflow compiled.")

### Step 5: Stream an interaction
Execute the graph with the tracer callback to observe emitted spans for the model and tool steps in real time.


In [ ]:
config = {"configurable": {"thread_id": "1"}, "callbacks": [TRACER]}
input_message = HumanMessage(content="Can you play Taylor Swift's most popular song?")

for event in APP.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()